We are going to implement Monte Carlo Tree Search in this notebook

In [1]:
import os
os.chdir('../')
from game import Connect2Game

In [2]:
from itertools import combinations, permutations, combinations_with_replacement, product
import itertools
import numpy as np
import random
import copy

from abc import ABC, abstractmethod
    
class Game(ABC):
     
    @abstractmethod
    def play(self):
        pass
    @abstractmethod
    def _is_legal(self):
        pass
    @abstractmethod
    def _is_win(self):
        pass
    @abstractmethod
    def _get_possible_next_states(self):
        pass
    
class Connect2(Game):
    def __init__(self, state:list=[0,0,0,0] ):
        self.board=state
        self.action_space=[0,1,2,3]
        self.player=1
        self.reward_win = 1
        self.reward_draw = 0.5
        self.reward_loss = 0
        
        
    def reset(self,):
        self.board=[0,0,0,0]
        self.player=1
        return self
    
    def init_custom(self,state, player):
        self.board=state
        self.player=player
        return self

    def play(self, action):
      
        if action not in self.action_space:
            raise ValueError(f"Action: {action} is not allowed from the action space: {self.action_space}")
            
        if self._is_legal(action):
            self.board[action]= self.player
            is_win=self._is_win()
            if is_win:
                print("player", self.player, "is winner")
                return self.reward_win
            if not is_win and len(self._get_possible_actions(self.board))==0:
                print("draw")
                return self.reward_draw
            
            self.player = self.player*-1
            
        else:
            raise ValueError("Illegal move")
            
    def play_mcts(self, action):
        """From the perspective of player 1"""
      
        if action not in self.action_space:
            raise ValueError(f"Action: {action} is not allowed from the action space: {self.action_space}")
            
        if self._is_legal(action):
            self.board[action]= self.player
            is_win=self._is_win()
            if is_win and self.player==1:
                print("player", self.player, "is winner")
                return self.reward_win
            elif is_win and self.player==-1:
                print("player", self.player, "is winner")
                return self.reward_loss
            elif not is_win and len(self._get_possible_actions(self.board))==0:
                print("draw")
                return self.reward_draw
            
            self.player = self.player*-1
            
        else:
            raise ValueError("Illegal move")
        
    def _is_legal(self, action, state=None):
        """We allow to ways of calling the method. Directly to check if an state is win or lost or by chcking the board"""
        is_legal=False
        
        if state is None:
            state = self.board.copy()

        if state[action]==0:
            is_legal=True
        return is_legal

    def _is_win(self, state=None):
        """We allow to ways of calling the method. Directly to check if an state is win or lost or by chcking the board"""
        is_win = False
        if state is None:
            state = self.board.copy()
            
        if abs(sum(state[0:2]))==2 or abs(sum(state[1:3]))==2 or abs(sum(state[2:4]))==2:
            #print(self.player, "is winner")
            is_win = True
            return is_win
        


    def _get_possible_next_states(self, state):
        """Give possible states from the state position. Action is the same as index. Potentially problematic.
        We are passing current state as argument, not necessarily the state of the board as it is."""
        possible_actions = self._get_possible_actions(state=state)
        possible_next_states = []
        
        for action in possible_actions:
            board = state.copy()
            board[action]=self.player
            possible_next_states.append(board)
            
        return possible_next_states

        
    def _get_possible_actions(self, state): 
        """Return all posible actions in the form of array [1,2] for example for [1,0,0,-1]"""
        return np.where(np.array(state)==0)[0]
    
        
    def render(self,):
        print(self.board, "Next player:", self.player)


In [224]:
dict_aux ={"first_dict": {"second_dict": {"third_dict"}}}

In [3]:
game = Connect2()
game.reset()
print("Starting")
game.render()
game.play(0)
game.render()
print(game._get_possible_next_states(game.board))
game.render()
game.play(1)
game.render()
game.play(2)
game.render()
game.play(3)
game.render()



Starting
[0, 0, 0, 0] Next player: 1
[1, 0, 0, 0] Next player: -1
[[1, -1, 0, 0], [1, 0, -1, 0], [1, 0, 0, -1]]
[1, 0, 0, 0] Next player: -1
[1, -1, 0, 0] Next player: 1
[1, -1, 1, 0] Next player: -1
draw
[1, -1, 1, -1] Next player: -1


In [11]:
game._is_win(state=[1,1,0,-1])

True

In [4]:
def UCB(w, n, N, c=2):
    """
    w number of wins, n number of runs of the node, 
    N total number of simulations of the parent node, c exploration parameter
    """
    out = w/n*c*(math.log(N)/n)**(0.5)
    
def invert_state(state: list):
    """From the perspective of player 1"""
    if sum(np.array(state))!=0:
        state=[-x for x in state]
    return state

class Node:
    def __init__(self, state: list, parent=None, is_root=False, is_leaf=False, path_to_root=None):
        self.state = state
        self.is_root = is_root
        self.parent = parent
        self.is_leaf = is_leaf
        self.children = []
        self.ucp=0
        self.wins=0
        self.visits=0
        self.path_to_root=path_to_root
        
    def __repr__(self):
        return f"State:{self.state}, is_root={self.is_root},is_leaf={self.is_leaf}\n Visits:{self.visits}, Wins: {self.wins} \n" + f"children:{[children_node.state for children_node in self.children]}"
        
    def add_children(self, state):
        key_state = str(state)
        path_to_root = [self.path_to_root,key_state]
        self.children.append(Node(state=state, is_root=False, is_leaf=True, path_to_root=path_to_root))
        self.is_leaf=False
        
    def update_node(self, wins, visits):
        self.wins+=wins
        self.visits+=visits
        

    
class MonteCarloTreeSearch:
    """Always from the perspective of one player
    Start in state0 s0
        is a leaf node?
            no: select child that maximize UCB as current state (repeat)
            yes:
                
            
    
    
    
    """
    def __init__(self, game: Game, number_simulations=4):
        self.game = game
        self.number_simulations=number_simulations

    def run(self, root_state):
        """Main loop"""
        root = Node(state=root_state, is_root=True, is_leaf=True)
        for n in range(self.number_simulations):
            node = root
            if node.is_leaf and len(node.children)==0:
                if node.visits==0:
                    # rollout
                    outcome = self.rollout(node.state)
                    # update node
                    node = self.update_node_based_on_outcome(outcome, node)

                else:
                    print("here")
                    # for each available action add children and run rollout
                    possible_next_states = self.game._get_possible_next_states(state=node.state)
                    for next_state in possible_next_states:
                        node.add_children(state=next_state)
            else:
                #expand children
                pass
        
        return node
    
    def rollout(self,state):
        """Monte Carlo simulation for the state"""
        
        self.game.init_custom(state=invert_state(state), player=1)
        while True:
            random_action = random.choice(self.game._get_possible_actions(self.game.board))
            outcome = self.game.play_mcts(action=random_action)
            # the outcome is the reward values (1 for winning, 0.5 draw, 0 lose and None while playing)
            if outcome is not None:
                self.game.reset()
                break

        return outcome
    def update_node_based_on_outcome(self, outcome, node):
        """For now we only consider winning, not draw"""
        if outcome == self.game.reward_win:
            node.update_node(wins=1, visits=1)
        else:
            node.update_node(wins=0, visits=1)
            
        return node
        
    
        

        


In [5]:
mcts = MonteCarloTreeSearch(game=Connect2())
node=mcts.run(root_state=[0,0,0,0])

player 1 is winner
here


In [6]:
node

State:[1, 1, 0, -1], is_root=True,is_leaf=False
 Visits:1, Wins: 1 
children:[[1, 1, 1, -1]]

AttributeError: 'MonteCarloTreeSearch' object has no attribute 'root_state'

In [217]:
game = Connect2()
root_node = Node(state=[0,0,0,0], is_root=True)
mcts = MCTS(game=game, root_node=root_node)
mcts.run()

State of the leaf node
[0, -1, 0, 0] Next player: 1
Action taken:  3
[0, -1, 0, 1] Next player: 1
1
simulating
Action taken:  2
player 1 is winner
[0, 1, -1, -1] Next player: 1
2
we have a winner, player 1


In [218]:
mcts.root_node.children

{'[1, 0, 0, 0]': State:[1, 0, 0, 0], is_root=False,is_leaf=True
  Visits:0, Wins: 0 
 children:[],
 '[0, 1, 0, 0]': State:[0, 1, 0, 0], is_root=False,is_leaf=True
  Visits:1, Wins: 1 
 children:[],
 '[0, 0, 1, 0]': State:[0, 0, 1, 0], is_root=False,is_leaf=True
  Visits:0, Wins: 0 
 children:[],
 '[0, 0, 0, 1]': State:[0, 0, 0, 1], is_root=False,is_leaf=True
  Visits:0, Wins: 0 
 children:[]}

In [222]:
path_to_root=mcts.root_node.children['[0, 1, 0, 0]'].path_to_root

In [227]:
for key_state in path_to_root[1:]:
    print(mcts.root_node.children[key_state])

State:[0, 1, 0, 0], is_root=False,is_leaf=True
 Visits:1, Wins: 1 
children:[]


In [ ]:
mcts.root_node[]

In [346]:
leaf_node.children

AttributeError: 'NoneType' object has no attribute 'children'

In [347]:
mcts.nodes[str([0, 0, 0, 0])].children

{'[1, 0, 0, 0]': Key state:[1, 0, 0, 0], is_root=False,is_leaf=True,
 '[0, 1, 0, 0]': Key state:[0, 1, 0, 0], is_root=False,is_leaf=True,
 '[0, 0, 1, 0]': Key state:[0, 0, 1, 0], is_root=False,is_leaf=True,
 '[0, 0, 0, 1]': Key state:[0, 0, 0, 1], is_root=False,is_leaf=True}

In [143]:
## explore game of connect2
game = Connect2Game()
print(game.get_next_state(board=[0, 0,0,0], player= 1, action=1))
print(game.get_next_state(board=[0, 0,0,0], player= -1, action=1))

(array([0, 1, 0, 0]), -1)
(array([ 0, -1,  0,  0]), 1)


In [146]:
game.get_valid_moves(board=[0,0,0,0])

[1, 1, 1, 1]

In [58]:
possible_values=[-1,0,1]
state=[0, 0,0,0]

In [ ]:
def _get_possible_next_states(state):
    

In [139]:
[list(i) for i in product(possible_values, repeat=len(state))]

[[-1, -1, -1, -1],
 [-1, -1, -1, 0],
 [-1, -1, -1, 1],
 [-1, -1, 0, -1],
 [-1, -1, 0, 0],
 [-1, -1, 0, 1],
 [-1, -1, 1, -1],
 [-1, -1, 1, 0],
 [-1, -1, 1, 1],
 [-1, 0, -1, -1],
 [-1, 0, -1, 0],
 [-1, 0, -1, 1],
 [-1, 0, 0, -1],
 [-1, 0, 0, 0],
 [-1, 0, 0, 1],
 [-1, 0, 1, -1],
 [-1, 0, 1, 0],
 [-1, 0, 1, 1],
 [-1, 1, -1, -1],
 [-1, 1, -1, 0],
 [-1, 1, -1, 1],
 [-1, 1, 0, -1],
 [-1, 1, 0, 0],
 [-1, 1, 0, 1],
 [-1, 1, 1, -1],
 [-1, 1, 1, 0],
 [-1, 1, 1, 1],
 [0, -1, -1, -1],
 [0, -1, -1, 0],
 [0, -1, -1, 1],
 [0, -1, 0, -1],
 [0, -1, 0, 0],
 [0, -1, 0, 1],
 [0, -1, 1, -1],
 [0, -1, 1, 0],
 [0, -1, 1, 1],
 [0, 0, -1, -1],
 [0, 0, -1, 0],
 [0, 0, -1, 1],
 [0, 0, 0, -1],
 [0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, -1],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 1, -1, -1],
 [0, 1, -1, 0],
 [0, 1, -1, 1],
 [0, 1, 0, -1],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 1, -1],
 [0, 1, 1, 0],
 [0, 1, 1, 1],
 [1, -1, -1, -1],
 [1, -1, -1, 0],
 [1, -1, -1, 1],
 [1, -1, 0, -1],
 [1, -1, 0, 0],
 [1, -1, 0, 1],
 [1, -1,

In [119]:

# Python program to print all
# the possible combinations
  
from itertools import permutations
  
# Get all combination of [1, 2, 3]
# of length 3
comb = permutations([1, 2, 3], )
  
for i in comb:
    print(i)